In [1]:
import os
import pandas as pd
from collections import Counter
import csv

In [2]:
gradebook = pd.read_csv('C148_Spring_2022_Spring_2022_grades.csv')

In [3]:
## replace NAs with zeroes
gradebook = gradebook.fillna(0)

In [4]:
## parse quiz grades, exported from bcourses

quiz_list = os.listdir('quizzes')

In [5]:
attempts_all_students = []

quiz_scores = []

quiz_max_total = 40 ## 23 quizzes times 2 minus 6 drops

for sid in gradebook['SID']:
    attempts_per_student = []
    for quiz in quiz_list:
        df = pd.read_csv('quizzes/'+quiz, dtype='string')
        counter = Counter(df['sis_id'].tolist())
        attempts = 0
        ## basically just counting to see how many times each SID shows up in each quiz
        for key in counter:
            if str(sid) in key:
                attempts = counter[key]
        if attempts >= 2:
            attempts_per_student.append(2)
        else:
            attempts_per_student.append(attempts)
    attempts_all_students.append(attempts_per_student)
    if sid == 3035932977: # drop first 10 quizzes for Ken Chen 3035932977
        quiz_score_per_student = sum(attempts_per_student)/26
    elif sid == 3036743721: # drop an extra quiz for Owen Ong 3036743721
        quiz_score_per_student = sum(attempts_per_student)/39
    else:
        quiz_score_per_student = sum(attempts_per_student)/quiz_max_total
    quiz_scores.append(min([quiz_score_per_student, 1.0]))

In [6]:
gradebook['Quiz Attempts'] = attempts_all_students
gradebook['Quiz Grade'] = quiz_scores

In [8]:
## i clicker
iclicker = pd.read_csv('iClicker_GradesExport_Reef_05-05-22.csv', index_col=1)

iclicker_percent = []
iclicker_final_score = []

for email in gradebook['Email']:
    try:
        iclicker_score = iclicker.loc[email]['Total %']
    except KeyError:
        iclicker_score = '0%'
    iclicker_score = float(iclicker_score[:-1])/100
    iclicker_percent.append(iclicker_score)
    if email == 'ken_chen@berkeley.edu': # drop clickers for the first few weeks
        if iclicker_score >= 0.46:
            iclicker_final_score.append(1)
        else:
            iclicker_final_score.append(0)
    elif iclicker_score >= 0.7:
        iclicker_final_score.append(1)
    else:
        iclicker_final_score.append(0)

In [9]:
gradebook['iClicker Percent'] = iclicker_percent
gradebook['iClicker Final Score'] = iclicker_final_score

In [10]:
## calculate problem set score with drops

## calculate percentages
gradebook['Problem Set 1 Percent'] = gradebook['Problem Set 1'] / gradebook['Problem Set 1 - Max Points']
gradebook['Problem Set 2 Percent'] = gradebook['Problem Set 2'] / gradebook['Problem Set 2 - Max Points']
gradebook['Problem Set 3 Percent'] = gradebook['Problem Set 3'] / gradebook['Problem Set 3 - Max Points']
gradebook['Problem Set 4 Percent'] = gradebook['Problem Set 4'] / gradebook['Problem Set 4 - Max Points']
gradebook['Problem Set 5 Percent'] = gradebook['Problem Set 5'] / gradebook['Problem Set 5 - Max Points']
gradebook['Problem Set 6 Percent'] = gradebook['Problem Set 6'] / gradebook['Problem Set 6 - Max Points']
gradebook['Problem Set 7 Percent'] = gradebook['Problem Set 7'] / gradebook['Problem Set 7 - Max Points']
gradebook['Problem Set 8 Percent'] = gradebook['Problem Set 8'] / gradebook['Problem Set 8 - Max Points']
gradebook['Problem Set 9 Percent'] = gradebook['Problem Set 9'] / gradebook['Problem Set 9 - Max Points']
problem_set_final_grade = []

for index, row in gradebook.iterrows():
    all_ps_scores = [
        row['Problem Set 1 Percent'],
        row['Problem Set 2 Percent'],
        row['Problem Set 3 Percent'],
        row['Problem Set 4 Percent'],
        row['Problem Set 5 Percent'],
        row['Problem Set 6 Percent'],
        row['Problem Set 7 Percent'],
        row['Problem Set 8 Percent'],
        row['Problem Set 9 Percent']
    ]
    if row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # drop ps 1, 2 and 3
        del all_ps_scores[0:3]
        all_ps_scores = [score * (1/6) for score in all_ps_scores]
    else: # drop 2 lowest PS
        all_ps_scores.remove(min(all_ps_scores))
        all_ps_scores.remove(min(all_ps_scores))
        all_ps_scores = [score * (1/7) for score in all_ps_scores]
    problem_set_final_grade.append(sum(all_ps_scores))
gradebook['Problem Sets Final Grade'] = problem_set_final_grade

In [11]:
# calculate exam percentages
gradebook['Exam 1 Percent'] = gradebook['Exam 1'] / gradebook['Exam 1 - Max Points']
gradebook['Exam 2 Percent'] = gradebook['Exam 2'] / gradebook['Exam 2 - Max Points']
gradebook['Exam 3 Percent'] = gradebook['Exam 3'] / gradebook['Exam 3 - Max Points']
gradebook['Exam 4 Percent'] = gradebook['Exam 4'] / gradebook['Exam 4 - Max Points']
gradebook['Final Exam Percent'] = gradebook['Final Exam'] / gradebook['Final Exam - Max Points']

In [11]:
## now calculate final grade with scaling factors

In [12]:
ps_scaling_factor = 0.14
exam_1_scaling_factor = 0.11
exam_2_scaling_factor = 0.17
exam_3_scaling_factor = 0.17
exam_4_scaling_factor = 0.2
final_scaling_factor = 0.28
scaling_factors = [ps_scaling_factor, exam_1_scaling_factor,
                  exam_2_scaling_factor, exam_3_scaling_factor, 
                  exam_4_scaling_factor, final_scaling_factor]

In [13]:
final_grades = []

for index, row in gradebook.iterrows():
    final_percentages = [
        row['Problem Sets Final Grade'],
        row['Exam 1 Percent'],
        row['Exam 2 Percent'],
        row['Exam 3 Percent'],
        row['Exam 4 Percent'],
        row['Final Exam Percent']
    ]
    # get index of lowest score
    min_index=final_percentages.index(min(final_percentages))
    individual_scaling_factors = scaling_factors.copy()
    individual_scaling_factors[min_index] = round(individual_scaling_factors[min_index]-0.1,2)
    final_percentages.append(row['Quiz Grade'])
    final_percentages.append(row['iClicker Final Score'])
    individual_scaling_factors.append(0.015)
    individual_scaling_factors.append(0.015)
    # forced drops for some students
    if row['First Name'] == 'Crystal' and row['Last Name'] == 'Le': # exam 4
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.17, 0.1, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Weilin' and row['Last Name'] == 'Zhang': # exam 4
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.17, 0.1, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Albert' and row['Last Name'] == 'Duenas': # exam 3
        individual_scaling_factors = [0.14, 0.11, 0.17, 0.07, 0.2, 0.28, 0.015, 0.015]
    elif row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # full drop exam 1
        individual_scaling_factors = [0.14, 0, 0.17, 0.17, 0.2, 0.28, 0.015, 0.015]
    ## multiply the two lists
    scaled_final_percentages = [a*b for a,b in zip(final_percentages,individual_scaling_factors)]
    final_grade = sum(scaled_final_percentages)
    if row['First Name'] == 'Ken' and row['Last Name'] == 'Chen': # to deal with missing 1%
        final_grade = final_grade * (100/99)
    final_grades.append(final_grade)

In [14]:
gradebook['Final Grade'] = final_grades

In [17]:
gradebook.to_csv('gradebook_PMBC148_grades.csv')

In [ ]:
## data exploration

In [16]:
gradebook[(gradebook['Final Grade'] < 0.6) & (gradebook['Final Grade'] > 0.1)]

,First Name,Last Name,SID,Email,section_name,Problem Set 1,Problem Set 1 - Max Points,Problem Set 1 - Submission Time,Problem Set 1 - Lateness (H:M:S),Problem Set 2,...,Problem Set 7 Percent,Problem Set 8 Percent,Problem Set 9 Percent,Problem Sets Final Grade,Exam 1 Percent,Exam 2 Percent,Exam 3 Percent,Exam 4 Percent,Final Exam Percent,Final Grade
3,Albert,Duenas,3036941765,albertduenas@berkeley.edu,mcellbi-c148-dis-102-in-person,29.0,42.5,2022-01-25 00:35:44 -0800,00:00:00,33.5,...,0.000000,0.000000,0.000000,0.237705,0.593750,0.288889,0.480952,0.581633,0.483871,0.442179
8,Andrea,Abarca Cevallos,3034999956,andreabarca@berkeley.edu,mcellbi-c148-dis-102-in-person,33.1,42.5,2022-01-25 04:21:14 -0800,00:00:00,29.5,...,0.766667,0.866071,0.618644,0.712357,0.390625,0.177778,0.433333,0.489796,0.459677,0.480979
19,Chris,Dong,3036177104,cxdong@berkeley.edu,mcellbi-c148-dis-102-in-person,33.0,42.5,2022-01-24 20:31:36 -0800,00:00:00,34.5,...,0.866667,0.982143,0.932203,0.918915,0.937500,0.955556,0.933333,0.000000,0.000000,0.555134
22,Crystal,Le,3035688694,crystle@berkeley.edu,plantbi-c148-dis-101-in-person,38.0,42.5,2022-01-25 02:35:51 -0800,00:00:00,35.5,...,0.966667,1.000000,0.991525,0.954561,0.390625,0.466667,0.516667,0.632653,0.520161,0.582684
